In [1]:
# Install pertinent libraries
!pip install gradio transformers torch scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 123.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
#Import libraries into your editor
import gradio as gr
import torch
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from scipy.special import softmax

In [4]:
#setup
model_path = "Hatoun/finetuned-distilroberta-base-opinrank-sentiment"

tokenizer = AutoTokenizer.from_pretrained(model_path)
#config = AutoConfig.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [9]:

def preprocess(text):
    # Preprocess text (username and link placeholders)
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

def sentiment_analysis(text):
    text = preprocess(text)

    # Tokenize the text
    inputs = tokenizer(text, return_tensors="pt", padding=True)

    # Make a prediction
    with torch.no_grad():
        outputs = model(**inputs)


    # Get the predicted class probabilities
    scores = torch.softmax(outputs.logits, dim=1).tolist()[0]

    # Map the scores to labels
    labels = ['Positive','Negative', 'Neutral']
    scores_dict = {label: score for label, score in zip(labels, scores)}

    return scores_dict

print(sentiment_analysis("The hotel was amazing and the staff was very friendly!"))

{'Positive': 0.12188226729631424, 'Negative': 0.0016808323562145233, 'Neutral': 0.8764368891716003}


In [21]:
# Customizing the interface
title = "🌟 Sentiment Analysis of NYC Hotel Reviews 🌟"
description = """
Analyze the sentiment of reviews about New York City hotels. Determine if a review is Positive, Neutral, or Negative.
Enjoy exploring hotel feedback!
"""


# Gradio Interface with custom CSS for styling
demo = gr.Interface(
    fn=sentiment_analysis,
    inputs=gr.Textbox(
        placeholder="Write your hotel review here...",
        label="Hotel Review"
    ),
    outputs=gr.Label(num_top_classes=3),
    title=title,
    description=description,
   allow_flagging="never" )

# Launch the app
demo.launch(share=True)


/usr/local/lib/python3.10/dist-packages/gradio/interface.py:399: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d5ececffd80a0e2129.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [22]:
!pip install huggingface_hub

In [25]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write

In [26]:
%%writefile requirements.txt
gradio
transformers
torch
scipy

Writing requirements.txt


In [27]:
%%writefile app.py
import gradio as gr
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from scipy.special import softmax

model_path = "Hatoun/finetuned-distilroberta-base-opinrank-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

def sentiment_analysis(text):
    text = preprocess(text)
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    scores = torch.softmax(outputs.logits, dim=1).tolist()[0]
    labels = ['Positive', 'Negative', 'Neutral']
    return {label: score for label, score in zip(labels, scores)}

title = "🌟 Sentiment Analysis of NYC Hotel Reviews 🌟"
description = "Analyze the sentiment of reviews about New York City hotels."

demo = gr.Interface(
    fn=sentiment_analysis,
    inputs=gr.Textbox(placeholder="Write your hotel review here...", label="Hotel Review"),
    outputs=gr.Label(num_top_classes=3),
    title=title,
    description=description,
    live=True,
    allow_flagging="never"
)

demo.launch()

Writing app.py
